In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms

from data import VQADataset, collate_fn_with_tokenizer
from transformers import BertTokenizer
from functools import partial

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
])

DATASET_ROOT = 'D:/VQA/easyVQA'
train_dataset = VQADataset(root_dir=DATASET_ROOT, 
                                split='train', 
                                transform=image_transform)

# 3. DataLoader 인스턴스 생성
# collate_fn tokenizer 버전 사용 유무에 따른 형식 차이
# (이전) {'image': ..., 'question': [...], 'answer': ...}
# (변경 후) {'image': ..., 'inputs': {'input_ids': ..., 'attention_mask': ...}, 'answer': ...}

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=partial(collate_fn_with_tokenizer, tokenizer=tokenizer)
)

# 4. DataLoader 테스트 (1배치 뽑아보기)
print("DataLoader에서 1배치 가져오기 테스트...")
try:
    first_batch = next(iter(train_loader))

    if first_batch:
        print(f"\n--- 첫 번째 배치 데이터 ---")
        
        # 이미지 배치
        img_batch = first_batch['image']
        print(f"Image 배치 타입: {type(img_batch)}")
        print(f"Image 배치 Shape: {img_batch.shape}") # (B, C, H, W) -> (2, 3, 224, 224)

        # 질문 배치
        q_batch = first_batch['inputs']
        print(f"\nQuestion 배치 타입: {type(q_batch)}")
        print(f"Question 배치 내용: {q_batch}") # ['what color is the shape?', 'what is the blue shape?']

        # 답변 배치
        ans_batch = first_batch['answer']
        print(f"\nAnswer 배치 타입: {type(ans_batch)}")
        print(f"Answer 배치 Shape: {ans_batch.shape}") # (B,) -> (2,)
        print(f"Answer 배치 내용: {ans_batch}") # [1, 0] (labels.txt 인덱스 기준)
        
        # 답변 인덱스를 다시 텍스트로 변환
        ans_texts = [train_dataset.idx_to_answer[idx] if idx >= 0 else 'UNK' for idx in ans_batch]
        print(f"Answer 텍스트 변환: {ans_texts}")

except Exception as e:
    print(f"\nDataLoader 테스트 중 오류 발생: {e}")

총 38575개의 샘플과 13개의 고유한 답변을 로드했습니다.
DataLoader에서 1배치 가져오기 테스트...

--- 첫 번째 배치 데이터 ---
Image 배치 타입: <class 'torch.Tensor'>
Image 배치 Shape: torch.Size([2, 3, 224, 224])

Question 배치 타입: <class 'transformers.tokenization_utils_base.BatchEncoding'>
Question 배치 내용: {'input_ids': tensor([[ 101, 2003, 2045, 2025, 1037, 9546, 1029,  102],
        [ 101, 2003, 2045, 1037, 3897, 4338, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

Answer 배치 타입: <class 'torch.Tensor'>
Answer 배치 Shape: torch.Size([2])
Answer 배치 내용: tensor([ 4, 12])
Answer 텍스트 변환: ['yes', 'no']
